# Import the relevant libraries

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
mnist_dataset

{'test': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 'train': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}

In [4]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='/Users/vijaygadre/tensorflow_datasets/mnist/3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

# Data Preprocessing

### Extract the training and testing data

In [5]:
mnist_train = mnist_dataset['train']
mnist_test = mnist_dataset['test']

### Generate Validation Data

In [6]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [7]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

### Scale the data

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(tf.autograph.experimental.do_not_convert(scale))
scaled_test = mnist_test.map(scale)

### Shuffle the data

In [9]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

### Batch the Data 

In [10]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Building up the Model (Feed-Forward NN)

In [11]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [12]:
model.compile(optimizer='ADAM', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(train_data, epochs=5, validation_data=(validation_inputs, validation_targets), verbose=1)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
527/540 [============================>.] - ETA: 0s - loss: 0.5142 - accuracy: 0.8536WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x164e26040> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
540/540 [==============================] - 3s 4ms/step - loss: 0.5080 - accuracy: 0.8553 - val_loss: 0.1428 - val_accuracy: 0.9593
Epoch 2/5
540/540 [==============================] - 2s 2ms/step - loss: 0.1155 - accuracy: 0.9643 - val_loss: 0.0918 - val_accuracy: 0.9737
Epoch 3/5
540/540 [==============================] - 1s 2ms/step - loss: 0.0797 - accuracy: 0.9745 - val_loss: 0.0660 - val_accuracy: 0.9803
Epoch 4/5
540/540 [==============================] - 1s 2ms/step - loss: 0.0563 - accuracy: 0.9821 - val_loss: 0.0580 - val_accuracy: 0.9817
Epoch 5/5
540/540 [==============================] - 1s 2ms/step - loss: 0.0456 - accuracy: 0.9853 - val_loss: 0.0354 - val_accur

In [14]:
test_loss, test_accuracy = model.evaluate(test_data)

print("Test Loss is: {0:.2f}\nTest Accuracy is: {1:.2f}".format(test_loss, test_accuracy*100.))

1/1 [==============================] - 0s 328ms/step - loss: 0.0713 - accuracy: 0.9798
Test Loss is: 0.07
Test Accuracy is: 97.98
